<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/08_Computational_Graph_Viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 8: Anatomia Grafu Obliczeniowego (DAG)

Kiedy wykonujesz operacje na tensorach z `requires_grad=True`, PyTorch nie tylko liczy wynik.
Buduje w pamięci drzewo (graf), które zapamiętuje **historię operacji**.

**Kluczowe pojęcia:**
1.  **Leaf Node (Liść):** Tensor stworzony przez użytkownika (np. Wagi, Dane). Nie ma historii (`grad_fn` jest None).
2.  **Root Node (Korzeń):** Wynik końcowy (np. Loss).
3.  **`grad_fn`:** Funkcja odwrotna. Jeśli zrobiłeś mnożenie (`Mul`), w grafie powstaje węzeł `MulBackward0`, który wie, jak policzyć pochodną mnożenia.

W tej lekcji prześledzimy ten graf ręcznie, cofając się od Wyniku do Wejścia.

In [1]:
import torch

# 1. TWORZYMY LIŚCIE (Wejście)
# requires_grad=True oznacza: "Zacznij śledzić historię od tego momentu"
a = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([3.0], requires_grad=True)

print(f"Czy 'a' jest liściem? {a.is_leaf}")
print(f"Czy 'a' ma grad_fn? {a.grad_fn}")  # None, bo to początek historii

Czy 'a' jest liściem? True
Czy 'a' ma grad_fn? None


## Budowanie Grafu (Forward Pass)

Wykonajmy proste działanie:
$$ c = a \times b $$
$$ d = c + 5 $$
$$ out = d \times 2 $$

Każda z tych operacji dodaje nowy węzeł do grafu.

In [2]:
# Krok 1: Mnożenie
c = a * b
print(f"c: {c}")
print(f"Funkcja tworząca c: {c.grad_fn}") 
# MulBackward0 -> Mówi: "Powstałem z mnożenia"

# Krok 2: Dodawanie
d = c + 5
print(f"Funkcja tworząca d: {d.grad_fn}")
# AddBackward0

# Krok 3: Wynik końcowy
out = d * 2
print(f"Funkcja tworząca out: {out.grad_fn}")
# MulBackward0

c: tensor([6.], grad_fn=<MulBackward0>)
Funkcja tworząca c: <MulBackward0 object at 0x00000200F474DAE0>
Funkcja tworząca d: <AddBackward0 object at 0x00000200F474DAE0>
Funkcja tworząca out: <MulBackward0 object at 0x00000200F474DAE0>


## Spacer po Grafie (Traversing the Graph)

Skoro `out` wie, że powstał z mnożenia `d * 2`, to musi mieć wskaźnik do `d`.
Możemy użyć metody `.next_functions`, żeby ręcznie cofnąć się w historii aż do `a` i `b`.

To jest dokładnie to, co robi silnik Autograd podczas `backward()`, tylko my zrobimy to "na piechotę".

In [3]:
print("--- ŚLEDZTWO WSTECZNE ---")

# Krok 0: Jesteśmy w 'out'
print(f"KROK 0 (Out): {out.grad_fn}")

# Krok 1: Z czego powstał out?
# next_functions zwraca listę krotek (funkcja, indeks)
parents = out.grad_fn.next_functions
print(f"KROK 1 (Rodzice Out): {parents}")
# Widzimy AddBackward0 (to jest nasze 'd'). 
# Drugi element to None (bo mnożyliśmy przez stałą '2', która nie ma historii)

# Wyciągamy funkcję, która stworzyła 'd'
d_fn = parents[0][0]

# Krok 2: Z czego powstało d?
grandparents = d_fn.next_functions
print(f"KROK 2 (Rodzice d): {grandparents}")
# Widzimy MulBackward0 (to jest nasze 'c')

# Wyciągamy funkcję, która stworzyła 'c'
c_fn = grandparents[0][0]

# Krok 3: Z czego powstało c?
great_grandparents = c_fn.next_functions
print(f"KROK 3 (Rodzice c): {great_grandparents}")
# Widzimy dwa obiekty AccumulateGrad.
# AccumulateGrad to "opakowanie" na nasze Liście (a i b).
# To tutaj gromadzą się gradienty.

--- ŚLEDZTWO WSTECZNE ---
KROK 0 (Out): <MulBackward0 object at 0x00000200F474E710>
KROK 1 (Rodzice Out): ((<AddBackward0 object at 0x00000200F474D450>, 0), (None, 0))
KROK 2 (Rodzice d): ((<MulBackward0 object at 0x00000200F474E710>, 0), (None, 0))
KROK 3 (Rodzice c): ((<AccumulateGrad object at 0x00000200F474F370>, 0), (<AccumulateGrad object at 0x00000200F474E320>, 0))


## Wizualizacja Własna (ASCII Tree)

Zamiast polegać na zewnętrznych bibliotekach (jak `torchviz`, który często sprawia problemy na Windowsie przez brak plików systemowych), zachowamy się jak inżynierowie i **napiszemy własne narzędzie**.

Stworzymy prostą funkcję rekurencyjną, która przejdzie po grafie (używając atrybutu `next_functions`, który odkryliśmy wcześniej) i wypisze go w formie drzewa tekstowego.

To rozwiązanie jest:
1.  **Lekkie:** Czysty Python, zero zależności.
2.  **Niezawodne:** Zadziała zawsze, nawet na serwerze bez graficznego interfejsu.
3.  **Edukacyjne:** Pokaże dokładnie strukturę `Mul` -> `Add` -> `Leaf`.

In [9]:
# Zamiast polegać na zewnętrznym programie, napiszmy własną funkcję rekurencyjną.
# To przechodzi po grafie i rysuje go w konsoli.

def print_graph(grad_fn, level=0):
    # Wcięcie dla wizualizacji poziomu
    indent = "    " * level
    
    # Nazwa funkcji (np. MulBackward0)
    name = grad_fn.__class__.__name__
    
    print(f"{indent}➡️ {name}")
    
    # Jeśli funkcja ma "rodziców" (next_functions), idziemy głębiej
    if hasattr(grad_fn, 'next_functions'):
        for parent, _ in grad_fn.next_functions:
            if parent is not None:
                print_graph(parent, level + 1)
            else:
                # To oznacza, że dotarliśmy do stałej lub tensora bez gradientu
                print(f"{indent}    🔹 (Stała / Brak historii)")
    
    # Jeśli to AccumulateGrad, to znaczy, że dotarliśmy do Liścia (Zmiennej)
    if "AccumulateGrad" in name:
        # Możemy spróbować wyciągnąć nazwę zmiennej, jeśli ją ma
        print(f"{indent}    🍃 To jest LIŚĆ (Leaf Node) - np. Wagi")

print("--- WIZUALIZACJA GRAFU (ASCII) ---")
print("ROOT (Wynik końcowy)")
print_graph(out.grad_fn)

--- WIZUALIZACJA GRAFU (ASCII) ---
ROOT (Wynik końcowy)
➡️ MulBackward0
    ➡️ AddBackward0
        ➡️ MulBackward0
            ➡️ AccumulateGrad
                🍃 To jest LIŚĆ (Leaf Node) - np. Wagi
            ➡️ AccumulateGrad
                🍃 To jest LIŚĆ (Leaf Node) - np. Wagi
        🔹 (Stała / Brak historii)
    🔹 (Stała / Brak historii)


## 🥋 Black Belt Summary

1.  **Graf jest dynamiczny:** Graf powstaje w momencie wykonywania operacji (np. `+`, `*`). Jeśli w kodzie masz `if x > 0: y = x * 2`, to graf zmienia swój kształt w każdej iteracji (Define-by-Run).
2.  **`grad_fn` to mapa:** Każdy tensor (oprócz liści) ma w plecaku mapę, jak wrócić do domu.
3.  **Liście (`is_leaf`):** To parametry (Wagi), które chcemy aktualizować. Tylko dla nich PyTorch gromadzi `.grad`.

W następnej lekcji zobaczymy, jak manipulować tą historią (odcinanie grafu i tryby inferencji).